In [1]:
import gradio as gr
import openai
import anthropic
import google.generativeai as genai
import requests
import json
import os
from typing import Dict, Any, Optional
import asyncio
from dotenv import load_dotenv

In [2]:
# load API
load_dotenv(override=True)

True

In [3]:
class CodeCommenter:
    def __init__(self):
        # Initialize API clients
        self.openai_client = None
        self.anthropic_client = None
        self.gemini_client = None
        
        # Load API keys from environment variables
        self.setup_clients()
    
    def setup_clients(self):
        """Initialize API clients with keys from environment variables"""
        try:
            # OpenAI
            openai_key = os.getenv('OPENAI_API_KEY')
            if openai_key:
                self.openai_client = openai.OpenAI(api_key=openai_key)
            
            # Anthropic
            anthropic_key = os.getenv('ANTHROPIC_API_KEY')
            if anthropic_key:
                self.anthropic_client = anthropic.Anthropic(api_key=anthropic_key)
            
            # Google Gemini
            gemini_key = os.getenv('GOOGLE_API_KEY')
            if gemini_key:
                genai.configure(api_key=gemini_key)
                self.gemini_client = genai.GenerativeModel('gemini-2.0-flash-exp')
                
        except Exception as e:
            print(f"Warning: Error setting up API clients: {e}")
    
    def create_comments_prompt(self, code: str, language: str) -> str:
        """Create a prompt for the LLM to add comments and docstrings"""
        return f"""Please add detailed and helpful comments and docstrings to the following {language} code. 
        
Guidelines:
1. Add comprehensive docstrings for functions, classes, and modules
2. Add inline comments explaining complex logic
3. Follow the commenting conventions for {language}
4. Maintain the original code structure and functionality
5. Make comments clear and professional
6. Don't change the actual code logic, only add comments
7. Do not add code markdown delimiters like ```python

Here's the code to comment:

{code}

Please return only the commented code without any additional explanation or markdown formatting."""

    def create_tests_prompt(self, code: str, language: str) -> str:
        """Create a prompt for the LLM to generate unit tests"""
        return f"""Please generate comprehensive unit tests for the following {language} code.
        
Guidelines:
1. Use appropriate testing framework for {language} (pytest for Python, JUnit for Java, etc.)
2. Create tests for all functions and methods
3. Include both positive and negative test cases
4. Test edge cases and error conditions
5. Use meaningful test names that describe what is being tested
6. Include setup and teardown methods if needed
7. Add mock objects for external dependencies (like database connections)
8. Do not add code markdown delimiters like ```python
9. Follow testing best practices for {language}

Here's the code to test:

{code}

Please return only the unit test code without any additional explanation or markdown formatting."""

    def create_combined_prompt(self, code: str, language: str) -> str:
        """Create a prompt for the LLM to add both comments and unit tests"""
        return f"""Please add detailed comments and docstrings to the following {language} code AND generate comprehensive unit tests for it.
        
For Comments:
1. Add comprehensive docstrings for functions, classes, and modules
2. Add inline comments explaining complex logic
3. Follow the commenting conventions for {language}
4. Don't change the actual code logic, only add comments

For Unit Tests:
1. Use appropriate testing framework for {language} (pytest for Python, JUnit for Java, etc.)
2. Create tests for all functions and methods
3. Include both positive and negative test cases
4. Test edge cases and error conditions
5. Add mock objects for external dependencies (like database connections)
6. Follow testing best practices for {language}

Structure your response as:
1. First, provide the original code with added comments and docstrings 
2. Then, provide the unit tests as a separate section
3. Do not add code markdown delimiters like ```python
4. The 2 separated portions of code, comments and unit test should be clearly demarcated by comments specifying the following section purpose

Here's the code:

{code}

Please return the commented code followed by the unit tests, clearly separated."""

    def call_openai(self, prompt: str, model: str = "gpt-4o-mini") -> str:
        """Make API call to OpenAI"""
        if not self.openai_client:
            return "Error: OpenAI API key not configured. Please set OPENAI_API_KEY environment variable."
        
        try:
            response = self.openai_client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": "You are a helpful coding assistant that adds detailed comments, docstrings, and generates unit tests for code."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=4000,
                temperature=0.1
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            return f"Error calling OpenAI API: {str(e)}"
    
    def call_anthropic(self, prompt: str, model: str = "claude-3-5-haiku-20241022") -> str:
        """Make API call to Anthropic Claude"""
        if not self.anthropic_client:
            return "Error: Anthropic API key not configured. Please set ANTHROPIC_API_KEY environment variable."
        
        try:
            response = self.anthropic_client.messages.create(
                model=model,
                max_tokens=4000,
                temperature=0.1,
                messages=[
                    {"role": "user", "content": prompt}
                ]
            )
            return response.content[0].text.strip()
        except Exception as e:
            return f"Error calling Anthropic API: {str(e)}"
    
    def call_gemini(self, prompt: str) -> str:
        """Make API call to Google Gemini"""
        if not self.gemini_client:
            return "Error: Google API key not configured. Please set GOOGLE_API_KEY environment variable."
        
        try:
            response = self.gemini_client.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    max_output_tokens=4000,
                    temperature=0.1,
                )
            )
            return response.text.strip()
        except Exception as e:
            return f"Error calling Gemini API: {str(e)}"
    
    def call_ollama(self, prompt: str, model: str = "llama3.2:latest") -> str:
        """Make API call to Ollama (local)"""
        try:
            url = "http://localhost:11434/api/generate"
            data = {
                "model": model,
                "prompt": prompt,
                "stream": False,
                "options": {
                    "temperature": 0.1,
                    "num_predict": 4000
                }
            }
            
            response = requests.post(url, json=data, timeout=60)
            if response.status_code == 200:
                result = response.json()
                return result.get('response', '').strip()
            else:
                return f"Error calling Ollama API: HTTP {response.status_code}"
        except requests.exceptions.ConnectionError:
            return "Error: Could not connect to Ollama. Make sure Ollama is running locally on port 11434."
        except Exception as e:
            return f"Error calling Ollama API: {str(e)}"

    def process_code(self, language: str, code: str, llm: str, generate_comments: bool, generate_tests: bool) -> str:
        """Process the given code based on selected options"""
        if not code.strip():
            return "Error: Please provide code to process."
        
        if not generate_comments and not generate_tests:
            return "Error: Please select at least one option (Generate comments or Generate test units)."
        
        # Determine which prompt to use
        if generate_comments and generate_tests:
            prompt = self.create_combined_prompt(code, language)
        elif generate_comments:
            prompt = self.create_comments_prompt(code, language)
        else:  # generate_tests only
            prompt = self.create_tests_prompt(code, language)
        
        # Route to appropriate LLM
        if llm == "gpt-4o-mini":
            return self.call_openai(prompt, "gpt-4o-mini")
        elif llm == "claude-3-5-haiku-20241022":
            return self.call_anthropic(prompt, "claude-3-5-haiku-20241022")
        elif llm == "gemini-2.0-flash":
            return self.call_gemini(prompt)
        elif llm == "ollama:llama3.2:latest":
            return self.call_ollama(prompt, "llama3.2:latest")
        else:
            return f"Error: Unsupported LLM: {llm}"

In [4]:
def create_gradio_interface():
    """Create and configure the Gradio interface"""
    commenter = CodeCommenter()
    
    # Define the main function for the interface
    def process_code_interface(language, code, llm, generate_comments, generate_tests):
        """Process the code and return processed version based on selected options"""
        if not code.strip():
            return "Please enter some code to process."
        
        if not generate_comments and not generate_tests:
            return "Please select at least one option: Generate comments or Generate test units."
        
        # Show processing message
        options = []
        if generate_comments:
            options.append("comments")
        if generate_tests:
            options.append("unit tests")
        
        processing_msg = f"Processing {language} code with {llm} to generate {' and '.join(options)}..."
        print(processing_msg)
        
        # Process the code
        result = commenter.process_code(language, code, llm, generate_comments, generate_tests)
        return result
    
    # Define default code
    default_code = """import pyodbc
from tabulate import tabulate
def connect_to_sql_server(server_name, database, username=None, password=None):
    try:
        if username and password:
            connection_string = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server_name};DATABASE={database};UID={username};PWD={password}"
        else:
            connection_string = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server_name};DATABASE={database};Trusted_Connection=yes"
        connection = pyodbc.connect(connection_string)
        print(f"Successfully connected to {server_name}/{database}")
        return connection
    except Exception as e:
        print(f"Failed to connect to {server_name}/{database}: {str(e)}")
        return None
def get_record_count(connection, table_name):
    try:
        cursor = connection.cursor()
        query = f"SELECT COUNT(*) FROM {table_name}"
        cursor.execute(query)
        count = cursor.fetchone()[0]
        cursor.close()
        print(f"Record count for {table_name}: {count}")
        return count
    except Exception as e:
        print(f"Failed to get record count for {table_name}: {str(e)}")
        return None
def select_top_records(connection, table_name, n):
    try:
        cursor = connection.cursor()
        query = f"SELECT TOP {n} * FROM {table_name}"
        cursor.execute(query)
        records = cursor.fetchall()
        columns = [column[0] for column in cursor.description]
        cursor.close()
        print(f"Top {n} records from {table_name}")
        if records:
            print(tabulate(records, headers=columns, tablefmt="grid"))
        return records
    except Exception as e:
        print(f"Failed to retrieve top {n} records from {table_name}: {str(e)}")
        return None
conn = connect_to_sql_server("localhost", "AdventureWorks_lite")
if conn:
    total_records = get_record_count(conn, "Sales.SalesOrderDetail")
    top_records = select_top_records(conn, "Production.Product", 10)
    conn.close()
    print("Connection closed successfully")"""

    css = """
textarea[rows]:not([rows="1"]) {
    overflow-y: auto !important;
    scrollbar-width: thin !important;
}
textarea[rows]:not([rows="1"])::-webkit-scrollbar {
    all: initial !important;
    background: #f1f1f1 !important;
}
textarea[rows]:not([rows="1"])::-webkit-scrollbar-thumb {
    all: initial !important;
    background: #a8a8a8 !important;
}
"""

    # Create the interface
    with gr.Blocks(title="Code Commenter & Test Generator", theme=gr.themes.Base(), css=css) as interface:
        gr.Markdown("# 🔧 Code Commenter & Test Generator")
        gr.Markdown("Add detailed comments, docstrings, and/or generate unit tests for your code using various LLM models.")
        
        with gr.Row():
            with gr.Column():
                code_input = gr.Textbox(
                    label="Input Code",
                    value=default_code,
                    lines=15,
                    max_lines=20,
                    info="Enter the code you want to process"
                )
            
            with gr.Column():
                code_output = gr.Textbox(
                    label="Processed Code",
                    lines=20,
                    max_lines=20,
                    info="Your code with added comments, docstrings, and/or unit tests"
                )
        
        # Add checkboxes below the textboxes
        with gr.Row():
            with gr.Column():
                generate_comments_checkbox = gr.Checkbox(
                    label="Generate comments",
                    value=True,
                    info="Add detailed comments and docstrings to the code"
                )
                generate_tests_checkbox = gr.Checkbox(
                    label="Generate test units",
                    value=False,
                    info="Generate comprehensive unit tests for the code"
                )
        
        with gr.Row():
            with gr.Column(scale=1):
                language_dropdown = gr.Dropdown(
                    choices=["Python", "Ruby", "Rust", "C++", "Java"],
                    value="Python",
                    label="Programming Language",
                    info="Select the programming language of your code"
                )
                
                llm_dropdown = gr.Dropdown(
                    choices=[
                        "gpt-4o-mini",
                        "claude-3-5-haiku-20241022", 
                        "gemini-2.0-flash",
                        "ollama:llama3.2:latest"
                    ],
                    value="gpt-4o-mini",
                    label="LLM Model",
                    info="Choose the language model to use"
                )
                
                generate_btn = gr.Button(
                    "🚀 Process Code", 
                    variant="primary",
                    size="lg"
                )
        
        # Add some API setup information
        gr.Markdown("## 📝 API Setup Instructions")
        gr.Markdown("""
        To use this tool, you need to set up API keys as environment variables:
        
        - **OpenAI**: Set `OPENAI_API_KEY`
        - **Anthropic**: Set `ANTHROPIC_API_KEY` 
        - **Google Gemini**: Set `GOOGLE_API_KEY`
        - **Ollama**: Make sure Ollama is running locally on port 11434
        """)
        
        gr.Markdown("## ✨ Features")
        gr.Markdown("""
        - **Generate Comments**: Add detailed docstrings and inline comments
        - **Generate Unit Tests**: Create comprehensive test suites with mocking for external dependencies
        - **Combined Mode**: Generate both comments and unit tests in one go
        - **Multiple LLMs**: Choose from OpenAI, Anthropic, Google Gemini, or local Ollama models
        - **Multiple Languages**: Support for Python, Ruby, Rust, C++, and Java
        """)
        
        # Connect the button to the processing function
        generate_btn.click(
            fn=process_code_interface,
            inputs=[language_dropdown, code_input, llm_dropdown, generate_comments_checkbox, generate_tests_checkbox],
            outputs=code_output,
            show_progress=True
        )
    
    return interface

In [5]:
print("🚀 Starting Code Commenter & Test Generator...")
print("📋 Setting up Gradio interface...")

# Create and launch the interface
interface = create_gradio_interface()

print("🌐 Launching interface...")
print("💡 The interface will open in your default browser")
print("🔧 Make sure to set up your API keys as environment variables")

# Launch with auto-opening in browser
interface.launch(
    server_name="127.0.0.1",
    server_port=7860,
    share=False,
    inbrowser=True,
    show_error=True
)

🚀 Starting Code Commenter & Test Generator...
📋 Setting up Gradio interface...
🌐 Launching interface...
💡 The interface will open in your default browser
🔧 Make sure to set up your API keys as environment variables
* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
